In [5]:
#!pip install -U googlemaps
#!pip install gmaps
#!pip install networkx
import requests
import json
import pandas as pd
import googlemaps
import gmaps
from ipywidgets.embed import embed_minimal_html

age=20
tko=5
days=5
budget=50000
gender=1
child=0
style=True
hBudget=3000
rBudget=500
profilePoi={'age':age,'been2tko':tko,'howlong':days,'budget':budget,
            'isFemale':gender,'with_child':child,'crampedSechedule':style}
profileHotel={'age':age,'been2tko':tko,'days':days,'budget':budget,'isFemale':gender,'hotel budget':hBudget}
profileRest={'age':age,'been2tko':tko,'days':days,'totalBudget':budget,'gender':gender, 'oneBudget':rBudget}
#request pois & hotel
req = requests.post('http://127.0.0.1:5007/',json = profilePoi)
req.encoding='utf-8'
req = req.json()
reqH = requests.post('http://127.0.0.1:5008/', json = profileHotel).json()
reqR = requests.post('http://127.0.0.1:5009/',json = profileRest).json()
#req = requests.post('http://127.0.0.1:5000/')
print(len(req),type(req),req,len(reqH),type(reqH),reqH, len(reqR),type(reqR),reqR,sep='\n')
# print(req.content.decode('utf-8'))

4
<class 'list'>
['[{"index":265,"name":"東京迪士尼海洋樂園 Tokyo DisneySea","rank":"排名第 1的Maihama觀光 (共5家)","address":"千葉縣浦安Maihama1-13279-0031","child":0.75,"latitude":35.62675,"longitude":139.885095,"score":94,"chosen":true}]', '[{"index":118,"name":"朝倉雕塑館 ASAKURA Museum of Sculpture, Taito","rank":"排名第 14的台東區觀光 (共560家)","address":"東京都台東區谷中7-18-10110-0001","child":0.0,"latitude":35.7269979,"longitude":139.7685625,"score":70,"chosen":true},{"index":237,"name":"相田光男美術館 Mitsuo Aida Museum","rank":"排名第 4的Marunouchi觀光 (共7家)","address":"東京都千代田區Marunouchi3-5-1 MarunouchiTokyo International Forum B1100-0005","child":0.0,"latitude":35.6767429,"longitude":139.7637532,"score":70,"chosen":true},{"index":107,"name":"陽光國際水族館 Sunshine Aquarium","rank":"排名第 7的豊島區觀光 (共179家)","address":"東京都豊島區3-1-3 HigashiikebukuroWorld Import Mart Bldg. Rooftop170-0013","child":0.5248449333,"latitude":35.7330243,"longitude":139.6966416,"score":114,"chosen":true},{"index":226,"name":" Nittere Plaza","rank":"排名第 50的港區觀光 (共626家)

In [13]:
for i in reqR:
    print(type(i),len(i),i['store_nm_jp'],i['lat'],i['lng'],sep='\n')

<class 'dict'>
23
鮨 太一
35.6710833
139.761152
<class 'dict'>
23
銀座 久兵衛 銀座本店
35.6685534
139.7611999
<class 'dict'>
23
とんかつ檍のカレー屋いっぺこっぺ
35.5604869
139.7171267
<class 'dict'>
23
韻松亭
35.7138661
139.7726182
<class 'dict'>
23
麺屋吉左右
35.6684815
139.8103305
<class 'dict'>
23
一番いちばん
35.5493955
139.4470043
<class 'dict'>
23
すず木
35.6702694
139.7725051
<class 'dict'>
23
銀座ふじやま
35.6731203
139.7655234
<class 'dict'>
23
そば処 和邑
35.7226253
139.7124928
<class 'dict'>
23
炭焼 金竜山
35.6453454
139.7282849


In [2]:
# set key to gmaps & googlemaps API
with open('../apiKey.txt') as f:
    api_key = f.readline()
    f.close
gmaps.configure(api_key=api_key)
gm = googlemaps.Client(key=api_key)
#get hotel pos. as the start & end point
hotel=(reqH['緯度'], reqH['經度'])

#read a day's POI as pandas df & loop
for d in range(len(req)):
    day=pd.read_json(req[d])

    # googlemaps direction API to determine order of POIs & output durations & distances
    waypoints=[]
    for i in range(len(day)):
        spot=(day['latitude'][i],day['longitude'][i])
        waypoints.append(spot)
    dOptRt=gm.directions(hotel,
                          hotel,
                          waypoints=waypoints,
                          optimize_waypoints=True, language='zh-TW')
    order = dOptRt[0]['waypoint_order']#list
    wpOrdered=[]
    for j in order:
        wpOrdered.append(waypoints[j])

    #print output
    routetext='讓您久等了，第'+str(d+1)+'天行程如下：\n'
    for k in range(len(day)):
        text = (day['name'][order[k]]+'約'+
                dOptRt[0]['legs'][k]['distance']['text']+'，景點間通勤約需'+
                dOptRt[0]['legs'][k]['duration']['text']+'\n')
        routetext+=text
    print(routetext)

    #generate the map for a day's scheduled 
#    fig = gmaps.figure()
 #   locations=[]
#    hover_text=[]
#    label=[]
#    for k in range(len(day)):
#        hover_text.append(day['name'][order[k]])
#        locations.append(wpOrdered[k])
#        label.append(str(k))
#    marker_layer = gmaps.marker_layer(
#        locations=locations,
#        hover_text=hover_text,
#        label=label
#    )
#    fig.add_layer(marker_layer)
#    fig
#    embed_minimal_html('../map/'+str(d)+'map.html', views=[fig])

讓您久等了，第1天行程如下：
參觀乃木神社 Nogi Shrine約9.7 公里，景點間通勤約需18 分鐘
參觀東京清真寺暨土耳其文化中心 Tokyo Camii & Turkish Culture Center約5.2 公里，景點間通勤約需16 分鐘
參觀井草八幡宮 Igusa Hachimangu Folklore Collection約10.5 公里，景點間通勤約需30 分鐘
參觀東京藝術劇場 Tokyo Metropolitan Theatre約14.5 公里，景點間通勤約需38 分鐘

讓您久等了，第2天行程如下：
參觀 Takashimaya Nihombashi約1.6 公里，景點間通勤約需6 分鐘
參觀東京海賊王塔 TOKYO ONE PIECE TOWER約5.0 公里，景點間通勤約需14 分鐘
參觀惠比壽啤酒紀念館 Museum of Yebisu Beer約4.6 公里，景點間通勤約需11 分鐘
參觀 Toneri Park約29.5 公里，景點間通勤約需36 分鐘

讓您久等了，第3天行程如下：
參觀深川江戶資料館 Fukagawa Edo Museum約4.3 公里，景點間通勤約需11 分鐘
參觀地鐵博物館 Subway Museum約13.2 公里，景點間通勤約需22 分鐘
參觀精工博物館 The Seiko Museum約15.4 公里，景點間通勤約需22 分鐘
參觀三社祭 Sanja Matsuri約2.7 公里，景點間通勤約需10 分鐘



In [3]:
day=pd.read_json(req[0])
day

,index,name,rank,address,child,latitude,longitude,score,chosen
0,210,東京藝術劇場 Tokyo Metropolitan Theatre,排名第 6的豊島區觀光 (共179家),東京都豊島區1-8-1 Nishiikebukuro171-0021,0.000000,35.729730,139.708040,70,True
1,201,井草八幡宮 Igusa Hachimangu Folklore Collection,排名第 4的杉並區觀光 (共215家),東京都杉並區善福寺1-33167-0041,0.000000,35.712755,139.597405,114,True
2,55,東京清真寺暨土耳其文化中心 Tokyo Camii & Turkish Culture Ce...,排名第 3的澀谷區觀光 (共279家),東京都澀谷區1-19 Oyamacho151-0065,0.025866,35.668191,139.676556,114,True
3,203,乃木神社 Nogi Shrine,排名第 2的Akasaka觀光 (共18家),東京都港區Akasaka8-11-27 Akasaka107-0052,0.000000,35.668314,139.728375,114,True


In [4]:
routetext_all=''
for d in range(len(req)):
    day=pd.read_json(req[d])
    #print output
    locations=[]
    hover_text=[]
    label=[]
    routetext=''
    for k in range(len(day)):
        text = (day['name'][k]+'\n大東京都會'+
                day['rank'][k]+'\n地址：'+
                day['address'][k]+'\n'*2)
        routetext+=text
        spot=(day['latitude'][k],day['longitude'][k])
        locations.append(spot)
        hover_text.append(day['name'][k])
        label.append(str(k+1))

    #generate the map for a day's scheduled 
    fig = gmaps.figure()
    marker_layer = gmaps.marker_layer(locations=locations,
                                      hover_text=hover_text,
                                      label=label
                                     )
    fig.add_layer(marker_layer)
    fig
    embed_minimal_html('../map/'+str(d)+'map.html', views=[fig])
    routetext_all+=routetext
routetext_all=routetext_all+'map.html'
print('---------------\n',routetext_all,'\n------------------')

---------------
 東京藝術劇場 Tokyo Metropolitan Theatre
大東京都會排名第 6的豊島區觀光 (共179家)
地址：東京都豊島區1-8-1 Nishiikebukuro171-0021

井草八幡宮 Igusa Hachimangu Folklore Collection
大東京都會排名第 4的杉並區觀光 (共215家)
地址：東京都杉並區善福寺1-33167-0041

東京清真寺暨土耳其文化中心 Tokyo Camii & Turkish Culture Center
大東京都會排名第 3的澀谷區觀光 (共279家)
地址：東京都澀谷區1-19 Oyamacho151-0065

乃木神社 Nogi Shrine
大東京都會排名第 2的Akasaka觀光 (共18家)
地址：東京都港區Akasaka8-11-27 Akasaka107-0052

惠比壽啤酒紀念館 Museum of Yebisu Beer
大東京都會排名第 12的澀谷區觀光 (共279家)
地址：東京都澀谷區惠比壽4-20-1

 Toneri Park
大東京都會排名第 2的足立區觀光 (共99家)
地址：東京都足立區1-1 Tonerikoen,121-0837

東京海賊王塔 TOKYO ONE PIECE TOWER
大東京都會排名第 16的港區觀光 (共626家)
地址：東京都港區4-2-8 ShibakoenTokyo Tower Foot Town105-0011

 Takashimaya Nihombashi
大東京都會排名第 24的中央區觀光 (共577家)
地址：東京都中央區2-4-1 Nihombashi103-8265

精工博物館 The Seiko Museum
大東京都會排名第 11的墨田區觀光 (共264家)
地址：東京都墨田區3-9-7 Higashi Mukojima131-0032

地鐵博物館 Subway Museum
大東京都會排名第 3的江戶川區觀光 (共191家)
地址：東京都江戶川區東葛西6-3-1134-0084

三社祭 Sanja Matsuri
大東京都會排名第 19的台東區觀光 (共560家)
地址：東京都台東區2-3-1 AsakusaAsakusa Shrine111-0032

深川江